In this notebook, we describe the random strategy. The idea is to split randomly the users into two groups of equal size.

In [1]:
N = 4 # number of antennas in the BS
K = 10 # number of users (K >= N)
X0 = 0.500 # parameter of the uniform distribution of the user on the x-axis
Y0 = 0.500 # parameter of the uniform distribution of the user on the y-axis
nsize = 10000 # size of the dataset

In [2]:
from itertools import product, combinations
import numpy as np
from numpy.linalg import inv
import h5py
import pandas as pd
from PIL import Image
from numpy import linalg as LA
import os
from sklearn.model_selection import train_test_split
from PIL import Image
from matplotlib.pyplot import imshow
import seaborn as sns
from sklearn import metrics
from IPython.display import clear_output
%matplotlib inline
np.random.seed(42) # fix the seed for reproducibility

In [3]:
def hermetian(H):
    return H.conj().T

In [4]:
def SNR1(H, P, sigma, i):
    K = H.shape[1]
    N = H.shape[0]
    P_i= P[i]
    h_i = H[:,i][:, np.newaxis]
    P = np.delete(P, i)
    set_minus_i = list(range(K))
    set_minus_i.pop(i)
    SNR = P_i*hermetian(h_i).dot(inv(H[:,set_minus_i].dot(np.diag(P)).dot(hermetian(H[:,set_minus_i])) + sigma**2*np.identity(N))).dot(h_i)
    return SNR.real

In [5]:
def SNR2(H, P, sigma, i, set_I1):
    K = H.shape[1]
    N = H.shape[0]
    P_i= P[i]
    set_0 = set_I1 + [i]
    h_i = H[:,i][:, np.newaxis]
    P = np.delete(P, set_0)
    set_minus_i = list(range(K))
    set_minus_i = [j for j in set_minus_i if j not in set_0]
    SNR = P_i*hermetian(h_i).dot(inv(H[:,set_minus_i].dot(np.diag(P)).dot(hermetian(H[:,set_minus_i])) + sigma**2*np.identity(N))).dot(h_i)
    return SNR.real

In [6]:
def sum_rate(df):
    sigma = 1
    with h5py.File('test_Data/' + df['H'], 'r') as hf:
        H = hf[df['H'][:-3]][:]
    with h5py.File('test_Data/' + df['P'], 'r') as hf:
        P = hf[df['P'][:-3]][:]
    P_diag = P.diagonal()
    N = H.shape[0]
    K = H.shape[1]
    cols = ['User'+str(i+1) for i in range(K)]
    conf = df[cols].values
    set_I1 = np.where(conf == 0)[0].tolist()
    set_I2 = np.where(conf == 1)[0].tolist()
    SNR_1 = [SNR1(H, P_diag, sigma, s) for s in set_I1] # SNR_i^(1) for i in I_1
    SNR_2 = [SNR2(H, P_diag, sigma, s, set_I1) for s in set_I2] # SNR_i^(2) for i in I_2
    R_1 = np.sum(np.log(1+np.array(SNR_1)))
    R_2 = np.sum(np.log(1+np.array(SNR_2)))
    return R_1 + R_2

In [7]:
cols = ['User' + str(i+1) for i in range(K)]

In [8]:
import random 
def partition(list_in, n):
    random.shuffle(list_in)
    l = [list_in[i::n] for i in range(n)]
    return l[0], l[1]

In [9]:
def random_DataSet(N, K, nsize):
    # creating empty dataframe with columns
    column_names=['User' + str(i+1) for i in range(K)]
    df = pd.DataFrame(columns = column_names, index = range(nsize))
    for j in range(nsize):
        
        clear_output(wait = True)
        
        list_in = [i for i in range(K)]
        # get the best clustering strategy
        set0, set1 = partition(list_in, 2)
        # transform it to one vector
        labels = np.zeros(K)
        for i in range(K):
            if  i in set0:
                labels[i] = 0
            else:
                labels[i] = 1
        
        df.loc[j] = list(labels)
        print('Progress:', np.round((j+1)/nsize*100,2), '%')
    
    #df.to_csv('random_data.csv', encoding='utf-8')
    return df

In [10]:
rdf = random_DataSet(N, K, 2000)

Progress: 100.0 %


In [11]:
test_df = pd.read_csv('test_data.csv')

In [12]:
test_df['sum_rate'] = test_df.apply(sum_rate, axis = 1)

In [13]:
rdf['H'] = test_df['H']
rdf['P'] = test_df['P']

In [14]:
rdf.head()

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,H,P
0,1,1,0,1,0,1,0,1,0,0,H_1.h5,P_1.h5
1,0,0,0,1,0,0,1,1,1,1,H_2.h5,P_2.h5
2,1,1,1,0,1,0,0,1,0,0,H_3.h5,P_3.h5
3,0,1,1,1,0,0,0,0,1,1,H_4.h5,P_4.h5
4,0,0,0,1,0,0,1,1,1,1,H_5.h5,P_5.h5


In [15]:
rdf['sum_rate'] = rdf.apply(sum_rate, axis = 1)

In [16]:
rdf.head()

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,H,P,sum_rate
0,1,1,0,1,0,1,0,1,0,0,H_1.h5,P_1.h5,22.083440
1,0,0,0,1,0,0,1,1,1,1,H_2.h5,P_2.h5,24.180543
2,1,1,1,0,1,0,0,1,0,0,H_3.h5,P_3.h5,20.090043
3,0,1,1,1,0,0,0,0,1,1,H_4.h5,P_4.h5,12.372057
4,0,0,0,1,0,0,1,1,1,1,H_5.h5,P_5.h5,14.824311


In [17]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(test_df['sum_rate'],rdf['sum_rate']))
mape = np.mean(np.abs((test_df['sum_rate'] - rdf['sum_rate']) / test_df['sum_rate'])) * 100
print('RMSE of the sum rate on the test set is:', round(rmse,2))
print('MAPE of the sum rate on the test set is:', round(mape,2))

RMSE of the sum rate on the test set is: 8.63
MAPE of the sum rate on the test set is: 31.4
